In [1]:
# Import dependencies
import subprocess
import json
from dotenv import load_dotenv
import os

from eth_account import Account
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI

In [2]:
# Load and set environment variables
load_dotenv()
mnemonic=os.getenv("mnemonic")
print(mnemonic)

uncle tobacco stone brisk release globe deer iron spare ill bounce youth stage owner issue


In [3]:
#Setup constants

from constants import *

In [4]:
# Import constants.py and necessary functions from bit and web3


from web3 import Web3
w3= Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))


#number of blocked mined
#w3.eth.blockNumber

#we want to get the balance
# w3.eth.getBalance("0x7b78e7B83AE10b5Bb000a28FD0ff11b406e37Fe9")

In [5]:
ETH = 'eth'
BTC = 'btc'
BTCTEST = 'btc-test'

In [6]:
def derive_wallets(phrase, coin, numderive):
    command = f'php derive -g --mnemonic="{phrase}" --numderive={numderive} --coin={coin} --format=json'
    p = subprocess.Popen(command,stdout=subprocess.PIPE,shell=True)
    (output, err) = p.communicate()
    p_status=p.wait()
    return json.loads(output)

In [7]:
coins = {'eth':derive_wallets(phrase=mnemonic,coin=ETH,numderive=3),'btc-test': derive_wallets(mnemonic,coin=BTCTEST,numderive=3)}
print(json.dumps(coins, indent=4, sort_keys=True))

{
    "btc-test": [
        {
            "address": "mhbPMTNRNg4aR82NmBDPvn28bUWUW9zd51",
            "index": 0,
            "path": "m/44'/1'/0'/0/0",
            "privkey": "cSyPcxTUeFxJcjqVmb5hupuGirjSBiqGVueXQE2NHmpeGvKXnH1E",
            "pubkey": "02f53ed1ae8a5ad518049cade7e41e187b276bd46f75c597727be04ce4ce20991c",
            "pubkeyhash": "16c6d5545d6e70c5dd9398828b22b95590e8208a",
            "xprv": "tprv8kBw7PjwVJXbKqt1sXPy9XaerHUtcSQmQjAfRMqykRzjAPvffL6i1UxKg5kEYmuJAD4xbXVJJMHctpPjvGWgFwfxtKeCUT6zXcXKUYV85Kb",
            "xpub": "tpubDGsyFonBdgDGDJuomB4ZYwEmRJzpmmbfz2mShstHAho7ztBSHivJByaBrDLJ3TkCkvP49HJE2D3DXicTykHGfsgJ4QL3ARMHW1NhPBkw34y"
        },
        {
            "address": "mfXhBW9nQBgrGYpRJMquVLjwFMy8FurX5T",
            "index": 1,
            "path": "m/44'/1'/0'/0/1",
            "privkey": "cPBLzRF8rha8jwnSyEmGhwZTbXKbpxXK8Jp22Ntg8QUv3hU1LHjA",
            "pubkey": "02c6db5cdf135209b3e0dbce561638909f90f0f85ab358a60163d356a825d104e4",
            "pubkeyh

In [8]:
eth_key = coins['eth'][0]['privkey']
btc_key = coins['btc-test'][0]['privkey']

In [9]:
def priv_key_to_account(coin, priv_key):
    global account
    if coin == 'eth':
        account=Account.privateKeyToAccount(priv_key)
    else:
        account=PrivateKeyTestnet(priv_key)
    return account

In [10]:
priv_key_to_account(ETH,eth_key)

In [11]:
priv_key_to_account(BTCTEST,btc_key)

<PrivateKeyTestnet: mhbPMTNRNg4aR82NmBDPvn28bUWUW9zd51>

In [12]:
def create_tx(coin,account,recipient,amount):
    global tx_data
    if coin =='eth':
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        tx_data= {
            "from": account.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
        }
        return tx_data
    else:
        tx_data = PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])
        return tx_data

In [13]:
create_tx(BTCTEST,account,'mfXhBW9nQBgrGYpRJMquVLjwFMy8FurX5T',0.0001)


'{"unspents":[{"amount":43706,"confirmations":311,"script":"76a91416c6d5545d6e70c5dd9398828b22b95590e8208a88ac","txid":"b4684d121fff9d79b914cdda5a619684a8dbd38480796799255b591d9b7a6318","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["mfXhBW9nQBgrGYpRJMquVLjwFMy8FurX5T",10000],["mhbPMTNRNg4aR82NmBDPvn28bUWUW9zd51",10654]]}'

In [14]:
def send_tx(coin, account, recipient, amount):
    if coin == ETH:
        tx = create_tx(coin,account, recipient, amount)
        signed_tx = account.sign_transaction(tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        tx_data = create_tx(coin,account,recipient,amount)
        signed = account.sign_transaction(tx_data)
        NetworkAPI.broadcast_tx_testnet(signed)
        return signed

In [15]:
send_tx(BTCTEST,account,'mhbPMTNRNg4aR82NmBDPvn28bUWUW9zd51',0.0001)

'010000000118637a9b1d595b259967798084d3dba88496615adacd14b9799dff1f124d68b4010000006a47304402201c4680e33cad09a093ab9a6b332e6362befc95a3f28a2c757b44613568659ff002205927155d0fc6c0f53125d421d8e8b1f14880fd1b9269f2746e7e8c60bb3c4a63012102f53ed1ae8a5ad518049cade7e41e187b276bd46f75c597727be04ce4ce20991cffffffff0210270000000000001976a91416c6d5545d6e70c5dd9398828b22b95590e8208a88ac9e290000000000001976a91416c6d5545d6e70c5dd9398828b22b95590e8208a88ac00000000'